# TODO:
* #### Tune XGBoost wihtout any features

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Utils

In [3]:
def plot_feature_importances(cols, feat_imps):
    fig = plt.figure(figsize = (15, 0.35*len(feat_imps)))
    feature_imp_df = pd.DataFrame(data=zip(cols, feat_imps), columns=["feature", "importance"]).sort_values(by="importance", ascending=False)
    plt.title('Feature importances', size=25, y=1.05)
    sns.barplot(data=feature_imp_df, x='importance', y='feature')
    plt.show()

# Loading Data

In [4]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [5]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feat Engineering

In [6]:
df['morbid'] = np.where(df.bmi>40,1,0)
df['obese'] = np.where(df.bmi>30,1,0)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,morbid,obese
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,1,1
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,0


In [7]:
def feature_risk_factors(df):
    df["risk_factors"] = df[[
        "avg_glucose_level", "age", "bmi", 
        "hypertension", "heart_disease", 
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 99 else 0) + \
        (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )
    return df

In [8]:
feature_risk_factors(df)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,morbid,obese,risk_factors
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,0,1
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,1,1,2
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,0,2
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25503,Female,27.0,0,0,No,Private,Urban,75.77,17.6,never smoked,0,0,0
25504,Male,49.0,0,0,Yes,Private,Urban,102.91,26.7,Unknown,0,0,3
25505,Female,3.0,0,0,No,children,Rural,104.04,18.3,Unknown,0,0,1
25506,Male,31.0,0,0,Yes,Private,Urban,82.41,28.7,never smoked,0,0,1


In [9]:
df = pd.get_dummies(df)
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,morbid,obese,risk_factors,gender_Female,gender_Male,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,1,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,1,2,1,0,...,0,1,0,0,1,0,1,0,0,0
3,56.0,0,0,64.87,28.8,0,0,2,0,1,...,0,1,0,0,0,1,0,0,1,0
4,24.0,0,0,73.36,28.8,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,0


In [10]:
sc = StandardScaler()
feats_to_scale = ["age", "avg_glucose_level", "bmi"]
df[feats_to_scale] = sc.fit_transform(df[feats_to_scale])

In [13]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

# Tuning XGBoost - with feature engineering

In [14]:
def objective(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.00001, 0.5),
        'subsample': trial.suggest_loguniform('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.00001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.00001, 1.0)
    }
    

    cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=1337)

    cv_scores = np.empty(8)
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            early_stopping_rounds=50,
            verbose=0,
        )
        y_preds = model.predict_proba(X_test)[:, 1]
        cv_scores[fold] = roc_auc_score(y_test, y_preds)
    
    auc = np.mean(cv_scores)
    print(f"AVG CV AUC: \t {auc}")

    return auc

In [15]:
study = optuna.create_study(study_name="xgboost_tuning", direction="maximize")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=100)

[I 2023-01-16 08:39:27,543] A new study created in memory with name: xgboost_tuning
[I 2023-01-16 08:39:34,965] Trial 0 finished with value: 0.8900998026034261 and parameters: {'n_estimators': 87, 'max_depth': 10, 'learning_rate': 0.028523547398637612, 'min_child_weight': 2, 'gamma': 0.08595428660869475, 'subsample': 0.693315689604206, 'colsample_bytree': 0.25352514809448096, 'reg_alpha': 0.00012805964720578848, 'reg_lambda': 0.01495725742459704}. Best is trial 0 with value: 0.8900998026034261.


AVG CV AUC: 	 0.8900998026034261


[I 2023-01-16 08:39:40,578] Trial 1 finished with value: 0.8881137584031583 and parameters: {'n_estimators': 464, 'max_depth': 7, 'learning_rate': 0.04092859536692872, 'min_child_weight': 8, 'gamma': 5.591033745535546e-05, 'subsample': 0.8956860909232143, 'colsample_bytree': 0.3140240610871242, 'reg_alpha': 0.03558386055319189, 'reg_lambda': 0.0029222443437971783}. Best is trial 0 with value: 0.8900998026034261.


AVG CV AUC: 	 0.8881137584031583


[I 2023-01-16 08:39:50,228] Trial 2 finished with value: 0.8889256035779854 and parameters: {'n_estimators': 332, 'max_depth': 5, 'learning_rate': 0.0632354806807259, 'min_child_weight': 3, 'gamma': 0.059611858244196445, 'subsample': 0.6918593401468862, 'colsample_bytree': 0.9107253124138736, 'reg_alpha': 0.0002045529704007026, 'reg_lambda': 0.0003352659271131514}. Best is trial 0 with value: 0.8900998026034261.


...

AVG CV AUC: 	 0.8902382735391964


[I 2023-01-16 08:49:45,754] Trial 98 finished with value: 0.8895605855638226 and parameters: {'n_estimators': 403, 'max_depth': 6, 'learning_rate': 0.17520049416366207, 'min_child_weight': 4, 'gamma': 0.31856517436672177, 'subsample': 0.7387806578622172, 'colsample_bytree': 0.37959233344024723, 'reg_alpha': 0.19187861661908057, 'reg_lambda': 8.456165736284847e-05}. Best is trial 73 with value: 0.8931405553331584.


AVG CV AUC: 	 0.8895605855638226


[I 2023-01-16 08:49:51,530] Trial 99 finished with value: 0.8890265450077992 and parameters: {'n_estimators': 429, 'max_depth': 4, 'learning_rate': 0.23840717723863875, 'min_child_weight': 4, 'gamma': 0.0026655103736747936, 'subsample': 0.6096577802784005, 'colsample_bytree': 0.32527864593592254, 'reg_alpha': 0.043192850011273286, 'reg_lambda': 3.533664249179831e-05}. Best is trial 73 with value: 0.8931405553331584.


AVG CV AUC: 	 0.8890265450077992


In [16]:
study.best_value

0.8931405553331584

In [17]:
study.best_params

{'n_estimators': 442,
 'max_depth': 3,
 'learning_rate': 0.16894325955966064,
 'min_child_weight': 4,
 'gamma': 2.3973788914112574e-05,
 'subsample': 0.7325220564292374,
 'colsample_bytree': 0.3440318161832336,
 'reg_alpha': 0.04070996761929491,
 'reg_lambda': 7.607115094642777e-05}

# Tuning LightGBM

In [18]:
from optuna.integration import LightGBMPruningCallback

def objective_lgbm(trial, X, y):
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
        "num_rounds": trial.suggest_int("num_rounds", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 1000),
        "lambda_l1": trial.suggest_loguniform('lambda_l1', 0.00001, 1.0),
        "lambda_l2": trial.suggest_loguniform('lambda_l2', 0.00001, 1.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction":  trial.suggest_loguniform('bagging_fraction', 0.2, 1.0),
        "feature_fraction": trial.suggest_loguniform('feature_fraction', 0.2, 1.0),
    }

    cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=1337)

    cv_scores = np.empty(8)
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", is_unbalance=True, **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            early_stopping_rounds=100,
            verbose=0,
            callbacks=[
                LightGBMPruningCallback(trial, "auc")
            ],  # Add a pruning callback
        )
        y_preds = model.predict_proba(X_test)[:, 1]
        cv_scores[fold] = roc_auc_score(y_test, y_preds)
    
    auc = np.mean(cv_scores)
    print(f"AVG CV AUC: \t {auc}")
    return auc

In [19]:
study_lgbm = optuna.create_study(direction="maximize", study_name="LGBM Tuning")
func = lambda trial: objective_lgbm(trial, X, y)
study_lgbm.optimize(func, n_trials=100, show_progress_bar=True)

[I 2023-01-16 09:10:18,070] A new study created in memory with name: LGBM Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0005855714597047802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0005855714597047802
[LightGBM] [Warning] bagging_fraction is set=0.6666434962571293, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6666434962571293
[LightGBM] [Warning] num_iterations is set=135, num_rounds=135 will be ignored. Current value: num_iterations=135
[LightGBM] [Warning] min_gain_to_split is set=6.311990548451227, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.311990548451227
[LightGBM] [Warning] lambda_l2 is set=0.002840373948516487, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.002840373948516487
[LightGBM] [Warning] feature_fraction is set=0.7047256722851812, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7047256722851812
[LightGBM] [Warning] min_data_in_leaf is set=595, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=595
[LightGBM] [Warning] lambda_l1 i

In [20]:
study_lgbm.best_value

0.8831792754303385

In [21]:
study_lgbm.best_params

{'n_estimators': 1130,
 'num_rounds': 370,
 'learning_rate': 0.22788987264312152,
 'num_leaves': 266,
 'max_depth': 12,
 'min_data_in_leaf': 738,
 'lambda_l1': 0.0008358004494648958,
 'lambda_l2': 0.0003991283244000063,
 'min_gain_to_split': 13.59458818477936,
 'bagging_fraction': 0.2859282837043752,
 'feature_fraction': 0.5430812019387112}

# Tuning RandomForestClassifier

In [22]:
def objective_rf(trial, X, y):
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_features": trial.suggest_int("max_features", 5, 15),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        "max_depth": trial.suggest_int("max_depth", 2, 20),
    }

    cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=1337)

    cv_scores = np.empty(8)
    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = RandomForestClassifier(**param_grid)
        model.fit(
            X_train,
            y_train
        )
        y_preds = model.predict_proba(X_test)[:, 1]
        cv_scores[fold] = roc_auc_score(y_test, y_preds)
    
    auc = np.mean(cv_scores)
    print(f"AVG CV AUC: \t {auc}")
    return auc

In [23]:
study_rf = optuna.create_study(direction="maximize", study_name="Random Forest Tuning")
func = lambda trial: objective_rf(trial, X, y)
study_rf.optimize(func, n_trials=100, show_progress_bar=True)

[I 2023-01-16 09:18:41,590] A new study created in memory with name: Random Forest Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

AVG CV AUC: 	 0.878260321908259
[I 2023-01-16 09:20:16,536] Trial 0 finished with value: 0.878260321908259 and parameters: {'n_estimators': 496, 'max_features': 12, 'min_samples_split': 17, 'min_samples_leaf': 4, 'max_depth': 16}. Best is trial 0 with value: 0.878260321908259.
AVG CV AUC: 	 0.8863679893157379
[I 2023-01-16 09:20:41,220] Trial 1 finished with value: 0.8863679893157379 and parameters: {'n_estimators': 172, 'max_features': 11, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_depth': 8}. Best is trial 1 with value: 0.8863679893157379.
AVG CV AUC: 	 0.8805103667711166
[I 2023-01-16 09:21:00,443] Trial 2 finished with value: 0.8805103667711166 and parameters: {'n_estimators': 343, 'max_features': 5, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_depth': 4}. Best is trial 1 with value: 0.8863679893157379.
AVG CV AUC: 	 0.8877928164211863
[I 2023-01-16 09:21:43,827] Trial 3 finished with value: 0.8877928164211863 and parameters: {'n_estimators': 323, 'max_features': 1

In [24]:
study_rf.best_value

0.8886577205527104

In [25]:
study_rf.best_params

{'n_estimators': 169,
 'max_features': 15,
 'min_samples_split': 4,
 'min_samples_leaf': 3,
 'max_depth': 5}